# 1 perpare minikube

## load environment variables

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/core
```

In [ ]:
source ${PJ_ROOT}/docs/environments/minikube/env

## start minikube

In [ ]:
export CPU_CORE_NUM="4"
export MEMORY_MB=8192
export K8S_VERSION="v1.14.1"
minikube start --cpus ${CPU_CORE_NUM} --memory ${MEMORY_MB} --kubernetes-version ${K8S_VERSION} --bootstrapper=kubeadm --extra-config=kubelet.authentication-token-webhook=true --extra-config=kubelet.authorization-mode=Webhook --extra-config=scheduler.address=0.0.0.0 --extra-config=controller-manager.address=0.0.0.0 --profile ${MINIKUBE_NAME}

expected)
```
😄  minikube v1.0.0 on darwin (amd64)
🤹  Downloading Kubernetes v1.14.1 images in the background ...
🔥  Creating virtualbox VM (CPUs=4, Memory=8192MB, Disk=20000MB) ...
💿  Downloading Minikube ISO ...
 142.88 MB / 142.88 MB [============================================] 100.00% 0s
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
🐳  Version of container runtime is 18.06.2-ce
⌛  Waiting for image downloads to complete ...
✨  Preparing Kubernetes environment ...
    ▪ kubelet.authentication-token-webhook=true
    ▪ kubelet.authorization-mode=Webhook
    ▪ scheduler.address=0.0.0.0
    ▪ controller-manager.address=0.0.0.0
🚜  Pulling images required by Kubernetes v1.14.1 ...
🚀  Launching Kubernetes v1.14.1 using kubeadm ... 
⌛  Waiting for pods: apiserver proxy etcd scheduler controller dns
🔑  Configuring cluster permissions ...
🤔  Verifying component health .....
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!
```

## check network configuration of minikube

#### for macOS

In [ ]:
NWNAME=$(VBoxManage showvminfo ${MINIKUBE_NAME} | grep "Host-only Interface" | awk 'match($0, /vboxnet[0-9]+/){print substr($0,RSTART,RLENGTH)}');echo ${NWNAME}

example)
```
vboxnet0
```

In [ ]:
export HOST_IPADDR=$(ifconfig ${NWNAME} | awk '/inet / {print $2}')
sed -i '' -e "s/<<HOST_IPADDR>>/${HOST_IPADDR}/" ${PJ_ROOT}/docs/environments/minikube/env
echo ${HOST_IPADDR}

example)
```
192.168.99.1
```

In [ ]:
NETMASK_HEX=$(ifconfig ${NWNAME} | awk '/netmask / {print $4}')
export NETMASK=$(echo "${NETMASK_HEX:2}" | perl -pe '$_ = unpack("B32", pack("H*", $_)); s/0+$//g; $_ = length')
echo ${NETMASK}

example)
```
24
```

#### for Ubuntu

In [ ]:
NWNAME=$(VBoxManage showvminfo ${MINIKUBE_NAME} | grep "Host-only Interface" | awk 'match($0, /vboxnet[0-9]+/){print substr($0,RSTART,RLENGTH)}');echo ${NWNAME}

example)
```
vboxnet0
```

In [ ]:
export HOST_IPADDR=$(ifconfig ${NWNAME}  | awk '/inet / {print $2}' | cut -d: -f2)
sed -i -e "s/<<HOST_IPADDR>>/${HOST_IPADDR}/" ${PJ_ROOT}/docs/environments/minikube/env
echo ${HOST_IPADDR}

example)
```
192.168.99.1
```

In [ ]:
NETMASK_IP=$(ifconfig ${NWNAME} | awk '/Mask/ {print $4}' | cut -d: -f2)
export NETMASK=$(ipcalc ${HOST_IPADDR} ${NETMASK_IP} | awk '/Netmask: / {print $4}')
echo ${NETMASK}

example)
```
24
```

## edit configuration of minikube

Edit `InsecureRegistry` of minikube in order to access local registory.

In [ ]:
if [ -f ${HOME}/.minikube/machines/${MINIKUBE_NAME}/config.json.org ]; then cp ${HOME}/.minikube/machines/${MINIKUBE_NAME}/config.json.org ${HOME}/.minikube/machines/${MINIKUBE_NAME}/config.json; else cp ${HOME}/.minikube/machines/${MINIKUBE_NAME}/config.json ${HOME}/.minikube/machines/${MINIKUBE_NAME}/config.json.org; fi
cat ${HOME}/.minikube/machines/${MINIKUBE_NAME}/config.json | perl -pse 's/"InsecureRegistry": \[/"InsecureRegistry": [\n                "$h\/$m",/g;' -- -h=${HOST_IPADDR} -m=${NETMASK} > /tmp/config.json
mv /tmp/config.json ${HOME}/.minikube/machines/${MINIKUBE_NAME}/config.json
diff -u ${HOME}/.minikube/machines/${MINIKUBE_NAME}/config.json.org ${HOME}/.minikube/machines/${MINIKUBE_NAME}/config.json

example)
```diff
--- /Users/nmatsui/.minikube/machines/${MINIKUBE_NAME}/config.json.org	2018-07-23 11:26:27.000000000 +0900
+++ /Users/nmatsui/.minikube/machines/${MINIKUBE_NAME}/config.json	2018-07-23 11:35:25.000000000 +0900
@@ -41,6 +41,7 @@
             "Env": null,
             "Ipv6": false,
             "InsecureRegistry": [
+                "192.168.99.1/24",
                 "10.96.0.0/12"
             ],
             "Labels": null,
             ```

## restart minikube

In [ ]:
minikube stop --profile ${MINIKUBE_NAME}
minikube start --cpus ${CPU_CORE_NUM} --memory ${MEMORY_MB} --kubernetes-version ${K8S_VERSION} --bootstrapper=kubeadm --extra-config=kubelet.authentication-token-webhook=true --extra-config=kubelet.authorization-mode=Webhook --extra-config=scheduler.address=0.0.0.0 --extra-config=controller-manager.address=0.0.0.0 --profile ${MINIKUBE_NAME}

example)
```
✋  Stopping "minikube" in virtualbox ...
🛑  "minikube" stopped.
😄  minikube v1.0.0 on darwin (amd64)
🤹  Downloading Kubernetes v1.14.1 images in the background ...
💡  Tip: Use 'minikube start -p <name>' to create a new cluster, or 'minikube delete' to delete this one.
🔄  Restarting existing virtualbox VM for "minikube" ...
⌛  Waiting for SSH access ...
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
🐳  Version of container runtime is 18.06.2-ce
⌛  Waiting for image downloads to complete ...
✨  Preparing Kubernetes environment ...
    ▪ kubelet.authentication-token-webhook=true
    ▪ kubelet.authorization-mode=Webhook
    ▪ scheduler.address=0.0.0.0
    ▪ controller-manager.address=0.0.0.0
🚜  Pulling images required by Kubernetes v1.14.1 ...
🔄  Relaunching Kubernetes v1.14.1 using kubeadm ... 
⌛  Waiting for pods: apiserver proxy etcd scheduler controller dns
📯  Updating kube-proxy configuration ...
🤔  Verifying component health ......
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!
```

### confirm nodes

In [ ]:
kubectl get nodes

example)
```
NAME       STATUS   ROLES    AGE    VERSION
minikube   Ready    master   6m6s   v1.14.1
```

## start local registry on hostOS

Edit `InsecureRegistry` of local docker daemon in order to accept minikube.

#### for macOS

**Docker for mac > Preference > Daemon > Advanced**

```diff
diff -u /tmp/docker-mac-configuration.json.org /tmp/docker-mac-configuration.json
--- /tmp/docker-mac-configuration.json.org	2018-05-15 09:41:37.000000000 +0900
+++ /tmp/docker-mac-configuration.json	2018-05-15 09:42:39.000000000 +0900
@@ -1,4 +1,7 @@
 {
   "debug" : true,
-  "experimental" : true
+  "experimental" : true,
+  "insecure-registries" : [
+    "192.168.99.0/24"
+  ]
 }
```

#### for Ubuntu

**edit /etc/docker/daemon.json**

```cat
sudo cat /etc/docker/daemon.json
{ "insecure-registries":["192.168.99.0/24"] }
```

In [ ]:
systemctl daemon-reload
systemctl restart docker

### start local registry

#### for macOS

In [ ]:
docker run --name registry -p 5000:5000 -d registry:2.7.1

In [ ]:
export REPOSITORY=${HOST_IPADDR}:5000
sed -i '' -e "s/<<REPOSITORY>>/${REPOSITORY}/" ${PJ_ROOT}/docs/environments/minikube/env
echo ${REPOSITORY}

#### for Ubuntu

In [ ]:
docker run --name registry -p 5000:5000 -d registry:2.7.1

In [ ]:
export REPOSITORY=${HOST_IPADDR}:5000
sed -i -e "s/<<REPOSITORY>>/${REPOSITORY}/" ${PJ_ROOT}/docs/environments/minikube/env
echo ${REPOSITORY}

### setup rbac

In [ ]:
kubectl apply -f rbac/tiller-rbac.yaml

In [ ]:
kubectl get serviceaccounts -n kube-system | grep tiller

expected)
```
tiller                               1         1m
```

In [ ]:
kubectl apply -f rbac/default-rbac.yaml

In [ ]:
kubectl get clusterroles | grep default-read

expected)
```
default-read                                                           1m
```

In [ ]:
kubectl get clusterrolebindings | grep default

expected)
```
default                                                2m
```

## setup helm

In [ ]:
helm version --client

expected)
```
Client: &version.Version{SemVer:"v2.13.1", GitCommit:"618447cbf203d147601b4b9bd7f8c37a5d39fbb4", GitTreeState:"clean"}
```

In [ ]:
helm init --service-account tiller

In [ ]:
helm repo update

In [ ]:
kubectl get pods --all-namespaces | grep tiller

expected)
```
kube-system   tiller-deploy-7ccf99cd64-gkn7t          1/1       Running   0          1m
```

In [ ]:
helm version

expected)
```
Client: &version.Version{SemVer:"v2.13.1", GitCommit:"618447cbf203d147601b4b9bd7f8c37a5d39fbb4", GitTreeState:"clean"}
Server: &version.Version{SemVer:"v2.13.1", GitCommit:"618447cbf203d147601b4b9bd7f8c37a5d39fbb4", GitTreeState:"clean"}
```